In [10]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from metadata import *
from data_manager import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
stack_fixed = 'MD662'
stack_moving = 'atlasV4'

warp_setting = 16
detector_id = 1

# Global Transform - between two annotation volumes

In [56]:
global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_global_alignment_parameters(stack_moving=stack_moving, moving_volume_type='annotation',
                                            stack_fixed=stack_fixed, fixed_volume_type='annotation',
                                            global_transform_scheme=global_transform_scheme)

In [84]:
vol_m = DataManager.load_annotation_volume(stack=stack_moving, downscale=32)

volume_m_alignedTo_f = \
transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                  xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

volume_m_alignedTo_f_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='annotation',
                                        stack_f=stack_fixed, type_f='annotation',
                                        downscale=32,
                                        global_transform_scheme=global_transform_scheme)

create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)

In [ ]:
display_volume_sections(volume_m_alignedTo_f)

# Global Transform Annotation Volumes

In [21]:
stack_fixed = 'MD589'
stack_moving = 'MD585'

In [22]:
warp_setting = 9

if warp_setting == 1:
    upstream_warp_setting = None
    transform_type = 'affine'
elif warp_setting == 7:
    upstream_warp_setting = None
    transform_type = 'rigid'
elif warp_setting == 8:
    upstream_warp_setting = None
    transform_type = None
elif warp_setting == 9:
    upstream_warp_setting = 8
    transform_type = 'rigid'
else:
    raise Exception('Warp setting not recognized.')

In [23]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      type_f='annotation_as_score',
                                      type_m='annotation_as_score',
                                      warp_setting=warp_setting,
                                      trial_idx=0)

In [24]:
# for structure in all_known_structures_sided_with_surround:
for structure in all_known_structures_sided:
# for structure in ['7N_L']:
    
    try:
        t = time.time()

        vol_m = DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                                          type_m='annotation_as_score', 
                                                          type_f='annotation_as_score',
                                                         warp_setting=upstream_warp_setting,
                                                         structure=structure)
        
        volume_m_alignedTo_f = \
        transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                          xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        volume_m_alignedTo_f_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                    type_m='annotation_as_score',
                                                    type_f='annotation_as_score',
                                                    warp_setting=warp_setting,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
        bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)
    
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t)) # 3s
    
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

Transform: 1.79 seconds.
Transform: 1.77 seconds.
Transform: 1.76 seconds.
Transform: 1.76 seconds.
Transform: 1.79 seconds.
Transform: 1.79 seconds.
Transform: 1.82 seconds.
Transform: 1.82 seconds.
Transform: 1.76 seconds.
Transform: 1.76 seconds.
Transform: 1.77 seconds.
Transform: 1.76 seconds.
Transform: 1.79 seconds.
Transform: 1.79 seconds.
Transform: 1.81 seconds.
Transform: 1.80 seconds.
Transform: 1.78 seconds.
Transform: 1.78 seconds.
Transform: 1.81 seconds.
Transform: 1.80 seconds.
Transform: 1.78 seconds.
Transform: 1.78 seconds.
Transform: 1.80 seconds.
Transform: 1.81 seconds.
Transform: 1.90 seconds.
Transform: 1.88 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.80 seconds.
Transform: 1.79 seconds.
Transform: 1.78 seconds.
Transform: 1.80 seconds.
Transform: 1.86 seconds.
Transform: 1.85 seconds.
Transform: 1.77 seconds.
Transform: 1.77 seconds.
Transform: 1.78 seconds.
Transform: 1.78 seconds.


# Global Transform

In [3]:
warp_setting = 16
trial_idx = None

In [4]:
warp_properties = registration_settings.loc[warp_setting]

upstream_warp_setting = warp_properties['upstream_warp_id']
if upstream_warp_setting == 'None':
    upstream_warp_setting = None
else:
    upstream_warp_setting = int(upstream_warp_setting)

In [5]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      prep_id_f=2,
                                      detector_id_f=detector_id,
                                      warp_setting=warp_setting)

In [6]:
# Parallel

In [8]:
def transform_volume_one_structure(structure):
    try:
        t = time.time()

        vol_m = DataManager.load_original_volume(stack=stack_moving, structure=structure, 
                                                 downscale=32)

        volume_m_alignedTo_f = \
        transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                          xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        volume_m_alignedTo_f_fp = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving,
                                                    stack_f=stack_fixed,
                                                    detector_id_f=detector_id,
                                                    warp_setting=warp_setting,
                                                    structure=structure,
                                                   trial_idx=trial_idx)

        create_parent_dir_if_not_exists(volume_m_alignedTo_f_fp)
        bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fp)
        
        upload_to_s3(volume_m_alignedTo_f_fp)
    
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t)) # 3s
    
    except Exception as e:
        sys.stderr.write('Error transforming volume %s: %s.\n' % (structure, e))

In [9]:
t = time.time()

pool = Pool(NUM_CORES)
pool.map(transform_volume_one_structure, all_known_structures_sided_with_surround)
pool.close()
pool.join()

sys.stderr.write('Transform all structures: %.2f seconds.\n' % (time.time() - t)) #

aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_10N_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_10N_L.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_4N_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_4N_L.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down3

Transform: 4.21 seconds.
Transform: 4.23 seconds.
Transform: 4.23 seconds.
Transform: 4.25 seconds.
Transform: 4.27 seconds.
Transform: 4.29 seconds.
Transform: 4.30 seconds.
Transform: 4.30 seconds.
Transform: 4.32 seconds.
Transform: 4.29 seconds.
Transform: 4.32 seconds.
Transform: 4.33 seconds.
Transform: 4.34 seconds.
Transform: 4.36 seconds.
Transform: 4.37 seconds.
Transform: 4.38 seconds.


aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_4N_L_surround_200.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_4N_L_surround_200.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_4N_R_surround_200.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_4N_R_surround_200.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlas

Transform: 4.12 seconds.
Transform: 4.16 seconds.
Transform: 4.19 seconds.
Transform: 4.19 seconds.
Transform: 4.15 seconds.
Transform: 4.17 seconds.
Transform: 4.20 seconds.
Transform: 4.19 seconds.
Transform: 4.26 seconds.
Transform: 4.22 seconds.
Transform: 4.19 seconds.
Transform: 4.23 seconds.
Transform: 4.24 seconds.
Transform: 4.21 seconds.
Transform: 4.31 seconds.
Transform: 4.46 seconds.


aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_Amb_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_Amb_L.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_Amb_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_Amb_R.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_dow

...

File does not exist: /shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_outerContour_surround_200.bp
Error transforming volume outerContour_surround_200: [Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_outerContour_surround_200.bp'.
File does not exist: /shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_sp5_surround_200.bp
Error transforming volume sp5_surround_200: [Errno 2] No such file or directory: '/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_sp5_surround_200.bp'.
Transform: 3.35 seconds.


aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_SC_surround_200.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_SC_surround_200.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_VCP_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_VCP_R.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16

Transform: 3.83 seconds.
Transform: 2.88 seconds.
Transform: 2.79 seconds.
Transform: 2.94 seconds.


aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_VCP_R_surround_200.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_VCP_R_surround_200.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_VLL_R_surround_200.bp" "s3://mousebrainatlas-data/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume_VLL_R_surround_200.bp"
aws s3 cp "/shared/CSHL_volumes/atlasV4/atlasV4_down32_scoreVolume_warp16_MD662_detector1_down32_scoreVolume/score_volumes/a

Transform: 2.70 seconds.
Transform: 2.67 seconds.
Transform: 2.69 seconds.
Transform all structures: 30.99 seconds.


In [ ]:
# Sequential

In [5]:
# Transform moving volume, sided, without surround

for structure in all_known_structures_sided_with_surround:
# for structure in all_known_structures_sided:
# for structure in ['7N_L']:
    
    try:
        t = time.time()

        vol_m = DataManager.load_original_volume(stack=stack_moving, structure=structure, downscale=32, volume_type='score')

        volume_m_alignedTo_f = \
        transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                          xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        volume_m_alignedTo_f_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                                    stack_f=stack_fixed, type_f='score',
                                                    downscale=32,
                                                    classifier_setting_m=classifier_setting,
                                                    classifier_setting_f=classifier_setting,
                                                    warp_setting=warp_setting,
                                                    structure=structure,
                                                   trial_idx=trial_idx)

        create_parent_dir_if_not_exists(volume_m_alignedTo_f_fn)
        bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)
        
        upload_from_ec2_to_s3(volume_m_alignedTo_f_fn)
    
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t)) # 3s
    
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

aws s3 cp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_10N_L.bp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_10N_L.bp


Child returned 0
0.46 seconds.
Transform: 2.88 seconds.


aws s3 cp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_10N_L_surround_200.bp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_10N_L_surround_200.bp


Child returned 0
0.47 seconds.
Transform: 2.91 seconds.


aws s3 cp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_10N_R.bp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_10N_R.bp


...

aws s3 cp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_Amb_R_surround_200.bp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_Amb_R_surround_200.bp


Child returned 0
0.41 seconds.
Transform: 2.82 seconds.


aws s3 cp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_DC_L.bp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_1_MD657_down32_scoreVolume_clf_30_trial_3_DC_L.bp


Child returned 0
0.36 seconds.
Transform: 2.81 seconds.


KeyboardInterrupt: 

# Read

In [6]:
warp_setting = 1
trial_idx = 0

In [7]:
# Read transformed volumes

structure = '7N_L'

warped_volume = DataManager.load_transformed_volume(stack_m=stack_moving, 
                                                                          stack_f=stack_fixed, 
                                    classifier_setting_m=classifier_setting,
                                    classifier_setting_f=classifier_setting,
                                    warp_setting=warp_setting,
                                    trial_idx=0, structure=structure)

In [ ]:
display_volume_sections(warped_volume)

# Structure-wise Transform

In [17]:
classifier_setting = 30
warp_setting = 5
trial_idx = 0

In [18]:
warp_properties = registration_settings.loc[warp_setting]
print warp_properties

upstream_warp_setting = warp_properties['upstream_warp_id']
if upstream_warp_setting == 'None':
    upstream_warp_setting = None
else:
    upstream_warp_setting = int(upstream_warp_setting)
    upstream_trial_idx = 1 # modify this for different upstream warp

upstream_warp_id                        1
transform_type                      rigid
terminate_thresh                    1e-06
grad_computation_sample_number     100000
grid_search_sample_number           10000
std_tx                                100
std_ty                                100
std_tz                                100
std_theta_xy                           30
surround_weight                   inverse
regularization_weight                   0
Name: 5, dtype: object


In [19]:
# if warp_setting == 1:
#     upstream_warp_setting = None
# elif warp_setting == 2:
#     upstream_warp_setting = 1
# elif warp_setting == 4:
#     upstream_warp_setting = 1
# elif warp_setting == 5:
#     upstream_warp_setting = 1
# else:
#     raise Exception('Warp setting not recognized.')

In [20]:
# for structure in all_known_structures_sided:
for structure in ['IC']:

    # Load local transform parameters
    
    try:

        t = time.time()
        
        local_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
        DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                              classifier_setting_m=classifier_setting,
                                              classifier_setting_f=classifier_setting,
                                              warp_setting=warp_setting,
                                              param_suffix=structure,
                                              trial_idx=trial_idx)

        # Read global tx
        global_transformed_moving_structure_vol = \
        DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            warp_setting=upstream_warp_setting, 
                                            trial_idx=upstream_trial_idx,
                                            structure=structure)

        # Transform
        local_transformed_moving_structure_vol = transform_volume(vol=global_transformed_moving_structure_vol, 
                                                 global_params=local_params, 
                                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                                 xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)

        # Save
        local_transformed_moving_structure_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                    classifier_setting_m=classifier_setting,
                                                    classifier_setting_f=classifier_setting,
                                                    warp_setting=warp_setting,
                                                    trial_idx=trial_idx,
                                                    structure=structure)

        create_parent_dir_if_not_exists(local_transformed_moving_structure_fn)
        bp.pack_ndarray_file(local_transformed_moving_structure_vol, local_transformed_moving_structure_fn)
        
        upload_from_ec2_to_s3(local_transformed_moving_structure_fn)
        
        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

aws s3 cp /shared/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_5_MD657_down32_scoreVolume_clf_30_trial_0/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_5_MD657_down32_scoreVolume_clf_30_trial_0_IC.bp s3://mousebrainatlas-data/CSHL_volumes/atlasV3/atlasV3_down32_scoreVolume_clf_30_warp_5_MD657_down32_scoreVolume_clf_30_trial_0/score_volumes/atlasV3_down32_scoreVolume_clf_30_warp_5_MD657_down32_scoreVolume_clf_30_trial_0_IC.bp


Child returned 0
0.46 seconds.
Transform: 5.00 seconds.


# Polyrigid Transform

In [3]:
warp_setting = 12
upstream_warp_setting = 1
classifier_setting = 2

In [4]:
globally_registered_volume = \
DataManager.load_volume_all_known_structures(stack_m=stack_moving, stack_f=stack_fixed,
                                            warp_setting=upstream_warp_setting,
                                            classifier_setting_m=classifier_setting,
                                            classifier_setting_f=classifier_setting,
                                            trial_idx=0)

File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0/score_volumes/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0_sp5.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0/score_volumes/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0_sp5.bp'
Score volume for sp5 does not exist.
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0/score_volumes/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_trial_0_outerContour.bp
[Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_1_MD642_down32_scoreVolume_clf_2_tri

In [5]:
structure_sizes = {k: np.count_nonzero(v) for k, v in globally_registered_volume.iteritems()}

total_size = sum(structure_sizes.values())
structure_sizes_percent = {s: float(size)/total_size for s, size in structure_sizes.iteritems()}

In [6]:
structure_covars = {}
for s, v in globally_registered_volume.iteritems():
    ys, xs, zs = np.where(v)
    nzs = np.c_[xs, ys, zs]
    nzsc = nzs - nzs.mean(axis=0)
    C = np.dot(nzsc.T, nzsc)/float(len(nzsc))
    S, V = np.linalg.eigh(C)
    structure_covars[s] = C

In [7]:
# Read Transform of each structure, do polyrigid transform

rigid_parameters = {}
anchor_points = {}

for structure in all_known_structures_sided:

    try:
        tx_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
        DataManager.load_alignment_parameters(stack_m=stack_moving,
                                          stack_f=stack_fixed,
                                          classifier_setting_f=2,  
                                          classifier_setting_m=2,
                                        warp_setting=5,
                                           trial_idx=0,
                                            param_suffix=structure)
        
        rigid_parameters[structure] = (tx_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f)
        anchor_points[structure] = centroid_m
    except:
        sys.stderr.write('Error loading transform parameters for %s.\n' % structure)

File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0_parameters_sp5.txt
Error loading transform parameters for sp5.
File does not exist: /oasis/projects/nsf/csd395/yuncong/CSHL_registration_parameters/atlasV2/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0/atlasV2_down32_scoreVolume_clf_2_warp_5_MD642_down32_scoreVolume_clf_2_trial_0_parameters_outerContour.txt
Error loading transform parameters for outerContour.


In [8]:
weights = structure_sizes_percent
alpha = 1.
sigmas = {s: alpha * v for s, v in structure_covars.iteritems()}

In [ ]:
# Parallel

def func(structure):
    
    polyrigid_registered_volume = transform_volume_polyrigid(globally_registered_volume[structure], 
                                  rigid_parameters.values(), 
                                  anchor_points.values(), 
                                  sigmas.values(), 
                                  weights.values())

    # Save
    fn = \
    DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                classifier_setting_m=classifier_setting,
                                                classifier_setting_f=classifier_setting,
                                                warp_setting=warp_setting,
                                                trial_idx=0,
                                                structure=structure)

    create_parent_dir_if_not_exists(fn)
    bp.pack_ndarray_file(polyrigid_registered_volume, fn)

In [ ]:
pool = Pool(15)
pool.map(func, globally_registered_volume.keys())
pool.terminate()
pool.join()

In [17]:
# Sequential

for structure, gv in globally_registered_volume.iteritems():
    print structure
    polyrigid_registered_volume = transform_volume_polyrigid(gv, 
                                  rigid_parameters.values(), 
                                  anchor_points.values(), 
                                  sigmas.values(), 
                                  weights.values())

    # Save
    fn = \
    DataManager.get_transformed_volume_filepath(stack_m=stack_moving, stack_f=stack_fixed,
                                                classifier_setting_m=classifier_setting,
                                                classifier_setting_f=classifier_setting,
                                                warp_setting=warp_setting,
                                                trial_idx=0,
                                                structure=structure)

    create_parent_dir_if_not_exists(fn)
    bp.pack_ndarray_file(polyrigid_registered_volume, fn)

LRt_R
Sp5C_L
Pn_L
7N_R
PBG_L
VLL_L
4N_R
VLL_R
7N_L
PBG_R
RtTg
Sp5C_R
Pn_R
LRt_L
Sp5O_R
SNR_L
12N
SNR_R
Sp5O_L
DC_R
VCA_L
3N_L
6N_R
Sp5I_L
RMC_R
5N_L
SNC_L
SNC_R
AP
RMC_L
Sp5I_R
3N_R
6N_L
IC
VCA_R
DC_L
Amb_R
VCP_R
LC_R
10N_R
7n_R
5N_R
Tz_L
Tz_R
7n_L
4N_L
LC_L
10N_L
SC
VCP_L
Amb_L


# Global Transform Reversed

In [4]:
warp_setting = 3
trial_idx = 0

In [6]:
if warp_setting == 1:
    upstream_warp_setting = None
elif warp_setting == 2:
    upstream_warp_setting = 1
elif warp_setting == 3:
    global_warp_setting = 1
    upstream_warp_setting = 2
else:
    raise Exception('Warp setting not recognized.')
    
if trial_idx in [0, 1]:
    upstream_trial_idx = 0

In [7]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      classifier_setting_m=classifier_setting,
                                      classifier_setting_f=classifier_setting,
                                      warp_setting=global_warp_setting,
                                      trial_idx=trial_idx)

In [ ]:
# for structure in ['PBG_L']:
for structure in all_known_structures_sided:
    
    try:
        
        t = time.time()

        local_transformed_moving_structure_vol = \
            DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                                classifier_setting_m=classifier_setting,
                                                classifier_setting_f=classifier_setting,
                                                warp_setting=upstream_warp_setting, 
                                                trial_idx=upstream_trial_idx,
                                                structure=structure)

        local_transformed_moving_structure_global_reversed_vol = \
        transform_volume_inverse(vol=local_transformed_moving_structure_vol, 
                                 global_params=global_params, 
                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                 xdim_m=xdim_m, ydim_m=ydim_m, zdim_m=zdim_m)

        local_transformed_moving_structure_global_reversed_fn = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                                        stack_f=stack_fixed, type_f='score',
                                                        downscale=32,
                                                        classifier_setting_m=classifier_setting,
                                                        classifier_setting_f=classifier_setting,
                                                        warp_setting=warp_setting,
                                                        structure=structure)

        create_if_not_exists(os.path.dirname(local_transformed_moving_structure_global_reversed_fn))
        bp.pack_ndarray_file(local_transformed_moving_structure_global_reversed_vol, local_transformed_moving_structure_global_reversed_fn)

        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('%s\n' % e)
        sys.stderr.write('Error transforming volume %s.\n' % structure)

# Global Transform Reversed (new)

In [18]:
warp_setting = 18

global_warp_setting = 16
local_warp_setting = 17
trial_idx = None

In [19]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m=stack_moving, stack_f=stack_fixed,
                                      prep_id_f=2,
                                      detector_id_f=detector_id,
                                      warp_setting=global_warp_setting)

In [ ]:
for structure in ['PBG_L']:
# for structure in all_known_structures_sided:
    
    try:
        
        t = time.time()

        local_transformed_moving_structure_vol = \
            DataManager.load_transformed_volume(stack_m=stack_moving, stack_f=stack_fixed,
                                                prep_id_f=2,
                                                detector_id_f=detector_id,
                                                warp_setting=local_warp_setting, 
                                                structure=structure)

        local_transformed_moving_structure_global_reversed_vol = \
        transform_volume_inverse(vol=local_transformed_moving_structure_vol, 
                                 global_params=global_params, 
                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                 xdim_m=xdim_m, ydim_m=ydim_m, zdim_m=zdim_m)

        local_transformed_moving_structure_global_reversed_fp = \
        DataManager.get_transformed_volume_filepath(stack_m=stack_moving,
                                                    stack_f=stack_fixed,
                                                    downscale=32,
                                                    prep_id_f=2,
                                                    detector_id_f=detector_id,
                                                    warp_setting=warp_setting,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(local_transformed_moving_structure_global_reversed_fp))
        bp.pack_ndarray_file(local_transformed_moving_structure_global_reversed_vol, local_transformed_moving_structure_global_reversed_fp)

        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('Error transforming volume %s: %s.\n' % (structure, e))

# Globally map specimen to atlas

In [25]:
# Load transform parameters

global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_alignment_parameters(stack_m='atlasV4', stack_f='MD662',
                                      prep_id_f=2,
                                      detector_id_f=detector_id,
                                      warp_setting=16)

In [ ]:
# for structure in ['PBG']:
for structure in all_known_structures:
    
    try:
        
        t = time.time()

        vol_f = DataManager.load_original_volume(stack='MD662',
                                                 downscale=32,
                                            prep_id=2,
                                            detector_id=detector_id,
                                            structure=structure)

        vol_f_reversed = transform_volume_inverse(vol=vol_f, 
                                                 global_params=global_params, 
                                                 centroid_m=centroid_m, centroid_f=centroid_f,
                                                 xdim_m=xdim_m, ydim_m=ydim_m, zdim_m=zdim_m)

        vol_f_reversed_fp = \
        DataManager.get_transformed_volume_filepath(stack_m='MD662',
                                                    stack_f='atlasV4',
                                                    prep_id_m=2,
                                                    detector_id_m=detector_id,
                                                    warp_setting=18,
                                                    structure=structure)

        create_if_not_exists(os.path.dirname(vol_f_reversed_fp))
        bp.pack_ndarray_file(vol_f_reversed, vol_f_reversed_fp)
        upload_to_s3(vol_f_reversed_fp)

        sys.stderr.write('Transform: %.2f seconds.\n' % (time.time() - t))
        
    except Exception as e:
        sys.stderr.write('Error transforming volume %s: %s.\n' % (structure, e))